# Title Page Element Identification

In [7]:
import pandas as pd

rubric_data = pd.read_excel('../../raw_data/Rubric_Style.xlsx', sheet_name='samples')
print(rubric_data.head())

   Unnamed: 0 NodeID                                  Document Criteria  \
0         NaN      1  Title Page (No title page, then -10%)\nThe tit...   
1         NaN    1.1                                           No Exist   
2         NaN    1.2                                           No Exist   
3         NaN    1.3                                           No Exist   
4         NaN    1.4                                           No Exist   

                             Document Clear Criteria  \
0  The Document should have title page. \n(No tit...   
1                        Existing Title Page Section   
2                  Existing Name of the organization   
3  Existing Name of the internee, Student ID and ...   
4  Existing Submission date of the internship report   

                   Simplifiezed version of Criteria        Criteria Title  \
0  *Existing Title Page Section\n*Existing Name o...           Title Page   
1                                            No Need      

In [9]:
print(rubric_data.isnull().sum())  # Count missing values per column


Unnamed: 0                                47
NodeID                                    19
Document Criteria                         18
Document Clear Criteria                   34
Simplifiezed version of Criteria          34
Criteria Title                            33
Answer Section/Subsection                 22
Complexity of Scoring                     34
Criteria Granuality                       34
Section Max Score an source               47
Influence On Scoring (Scoring Formula)    34
Type of Scording                          34
List of Subcondition Condition-Score       4
Unnamed: 13                               47
Unnamed: 14                               41
Unnamed: 15                               41
Rubric Subcriteria Topic                  26
Rubric Subcriteria                        30
Subcriteria condition                     44
Condition related score                   47
Unnamed: 20                               47
Unnamed: 21                               47
Unnamed: 2